In [1]:
import sys
sys.path.insert(0, '..')
import pandas as pd
import numpy as np
from config import *
from helper import _process_pred_labels
from evaluate import get_pred_labels

# DOWNLOAD DATA

In [2]:
train = pd.read_pickle('../data/prepared/train.pkl')
test = pd.read_pickle('../data/prepared/test.pkl')
target = pd.read_pickle('../data/prepared/target.pkl')

#### Replace \xa0

In [3]:
for text_col in TEXT_COLS:
    train[text_col] = train[text_col].str.replace('\xa0', ' ')
    test[text_col] = test[text_col].str.replace('\xa0', ' ')

#### Count puntuation

In [4]:
PUNCT_COLS = []
for text_col in TEXT_COLS:
    for punct in '!*.,?@$-_()':
        col_name = f'{text_col}_count_{punct}'
        train[col_name] = train[text_col].str.count(f'\{punct}')
        test[col_name] = test[text_col].str.count(f'\{punct}')
        PUNCT_COLS.append(col_name)

#### Len review

In [5]:
LEN_REVIEW_COLS = []
for text_col in TEXT_COLS:
    col_name = f'len_review_{text_col}'
    train[col_name] = train[text_col].str.split().str.len()
    test[col_name] = test[text_col].str.split().str.len()
    LEN_REVIEW_COLS.append(col_name)

In [6]:
#### Morphology

In [7]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [8]:
def lemmatize(string):
    normal_forms = []
    tags = []
    for word in string.split():
        p = morph.parse(word)[0]
        normal_forms.append(p.normal_form)
        tag = []
        if p.tag.POS:
            tag.append(p.tag.POS) 
        if p.tag.animacy:
            tag.append(p.tag.animacy)
        if p.tag.aspect:
            tag.append(p.tag.aspect)
        if p.tag.case:
            tag.append(p.tag.case)
        if p.tag.gender:
            tag.append(p.tag.gender)
        if p.tag.involvement:
            tag.append(p.tag.involvement)
        if p.tag.mood:
            tag.append(p.tag.mood)
        if p.tag.number:
            tag.append(p.tag.number)
        if p.tag.person:
            tag.append(p.tag.person)
        if p.tag.tense:
            tag.append(p.tag.tense)
        if p.tag.transitivity:
            tag.append(p.tag.transitivity)
        if p.tag.voice:
            tag.append(p.tag.voice)
        tags.append(' '.join(tag))
    return pd.Series({
        'normalized_string': ' '.join(normal_forms),
        'tags': ', '.join(tags)
        })

In [9]:
morph_train = []
morph_test = []
NORMALIZED_TEXT_COLS = []
TAG_COLS = []
for text_col in TEXT_COLS:
    norm_col = f'{text_col}_normalized_string'
    tag_col = f'{text_col}_tags'
    NORMALIZED_TEXT_COLS.append(norm_col)
    TAG_COLS.append(tag_col)

    mdf_train = train[text_col].apply(lemmatize)
    mdf_train.rename(columns={
        'normalized_string': norm_col,
        'tags': tag_col}, inplace=True)
    morph_train.append(mdf_train)

    mdf_test = test[text_col].apply(lemmatize)
    mdf_test.rename(columns={
        'normalized_string': norm_col,
        'tags': tag_col}, inplace=True)
    morph_test.append(mdf_test)

train = train.join(morph_train, how='left')
test = test.join(morph_test, how='left')


In [10]:
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_predict
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import *
from sklearn.compose import *
from sklearn.feature_extraction.text import *
from sklearn.svm import LinearSVC
from sklearn.base import clone
from sklearn.ensemble import *
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
import category_encoders as ce

In [11]:
EMBED_COLS = [col for col in train.columns if 'embed' in col]
PIPELINE_CHACE = '../.cache'

In [12]:
X_train, X_test, y_train, y_test = train_test_split(train, target, test_size=0.3, random_state=42)

In [13]:
base_model = LinearSVC()

In [14]:
char_features = ColumnTransformer(n_jobs=-1, transformers=[
    ('positive_char', TfidfVectorizer(analyzer='char', lowercase=False, dtype=np.float32), POSITIVE),
    ('negative_char', TfidfVectorizer(analyzer='char', lowercase=False, dtype=np.float32), NEGATIVE),
]
)
char_pipe = Pipeline(memory=PIPELINE_CHACE, steps=[
    ('char_features', char_features),
    ('char_model', clone(base_model))
])

In [15]:
word_features = ColumnTransformer(n_jobs=-1, transformers=[
    ('positive_word', TfidfVectorizer(analyzer='word', ngram_range=(1,3), dtype=np.float32), POSITIVE),
    ('negative_word', TfidfVectorizer(analyzer='word', ngram_range=(1,3), dtype=np.float32), NEGATIVE),
    ('position_word', TfidfVectorizer(analyzer='word', ngram_range=(1,3), dtype=np.float32), POSITION),
]
)
word_pipe = Pipeline(memory=PIPELINE_CHACE, steps=[
    ('word_features', word_features),
    ('word_model', clone(base_model))
])

In [16]:
word_features_normalized = ColumnTransformer(n_jobs=-1, transformers=[
    ('positive_normalized_word', CountVectorizer(analyzer='word', ngram_range=(1,3), dtype=np.float32), f'{POSITIVE}_normalized_string'),
    ('negative_normalized__word', CountVectorizer(analyzer='word', ngram_range=(1,3), dtype=np.float32), f'{NEGATIVE}_normalized_string'),
]
)
word_pipe_normalized = Pipeline(memory=PIPELINE_CHACE, steps=[
    ('word_normalized__features', word_features_normalized),
    ('word_normalized__model', clone(base_model))
])

In [17]:
tags_features = ColumnTransformer(n_jobs=-1, transformers=[
    ('positive_word', CountVectorizer(analyzer='word', ngram_range=(1,3), dtype=np.float32), f'{POSITIVE}_tags'),
    ('negative_word', CountVectorizer(analyzer='word', ngram_range=(1,3), dtype=np.float32), f'{NEGATIVE}_tags'),
]
)
tags_pipe = Pipeline(memory=PIPELINE_CHACE, steps=[
    ('word_features', tags_features),
    ('word_model', clone(base_model))
])

In [18]:
char_wb_features = ColumnTransformer(n_jobs=-1, transformers=[
    ('positive_char_wb', TfidfVectorizer(analyzer='char_wb', ngram_range=(1,3), dtype=np.float32), POSITIVE),
    ('negative_char_wb', TfidfVectorizer(analyzer='char_wb', ngram_range=(1,3), dtype=np.float32), NEGATIVE),
]
)
char_wb_pipe = Pipeline(memory=PIPELINE_CHACE, steps=[
    ('char_wb_features', char_wb_features),
    ('char_wb_model', clone(base_model))
])

In [19]:
embed_features = ColumnTransformer(n_jobs=-1, transformers=[
    ('embeddings', 'passthrough', EMBED_COLS)
]
)
embed_pipe = Pipeline(memory=PIPELINE_CHACE, steps=[
    ('char_wb_features', embed_features),
    ('char_wb_model', clone(base_model))
])

In [20]:
unordered_cat_features = ColumnTransformer(n_jobs=-1, transformers=[
    ('unordered_cat', OneHotEncoder(dtype=np.float32, handle_unknown='ignore'), UNORDERED_CATEGORIES)
]
)
unordered_cat_pipe = Pipeline(memory=PIPELINE_CHACE, steps=[
    ('unordered_cat_features', unordered_cat_features),
    ('unordered_cat_model', clone(base_model))
])

In [26]:
ordered_cat_features = ColumnTransformer(n_jobs=-1, transformers=[
    ('ordered_cat', 'passthrough', ORDERED_CATEGORIES)
]
)
ordered_cat_pipe = Pipeline(memory=PIPELINE_CHACE, steps=[
    ('ordered_cat_features', ordered_cat_features),
    ('ordered_cat_model', clone(base_model))
])

In [21]:
cat_encoders_features = ColumnTransformer(n_jobs=-1, transformers=[
    ('WOEEncoder', ce.WOEEncoder(UNORDERED_CATEGORIES+ORDERED_CATEGORIES), UNORDERED_CATEGORIES+ORDERED_CATEGORIES),
    ('TargetEncoder', ce.TargetEncoder(UNORDERED_CATEGORIES+ORDERED_CATEGORIES), UNORDERED_CATEGORIES+ORDERED_CATEGORIES),
    ('CountEncoder', ce.CountEncoder(UNORDERED_CATEGORIES+ORDERED_CATEGORIES), UNORDERED_CATEGORIES+ORDERED_CATEGORIES),
    ('GLMMEncoder', ce.GLMMEncoder(ORDERED_CATEGORIES), ORDERED_CATEGORIES)
]
)
cat_encoders_pipe = Pipeline(memory=PIPELINE_CHACE, steps=[
    ('unordered_cat_features', cat_encoders_features),
    ('unordered_cat_model', LGBMClassifier())
])

In [22]:
manual_features = ColumnTransformer(n_jobs=-1, transformers=[
    ('manual_features', 'passthrough', PUNCT_COLS+LEN_REVIEW_COLS)
]
)
manual_features_pipe = Pipeline(memory=PIPELINE_CHACE, steps=[
    ('manual_features_features', manual_features),
    ('manual_features_model', clone(base_model))
])

In [23]:
stacked_model = StackingClassifier(
    estimators=[
        ('char_pipe', char_pipe), 
        ('word_pipe', word_pipe),
        ('char_wb_pipe', char_wb_pipe),
        ('embed_pipe', embed_pipe),
        ('ordered_cat_pipe', ordered_cat_pipe),
        ('unordered_cat_pipe', unordered_cat_pipe),
        ('cat_encoders_pipe', cat_encoders_pipe),
        ('manual_features_pipe', manual_features_pipe),
        ('word_pipe_normalized', word_pipe_normalized),
        ('tags_pipe', tags_pipe)
    ],
    final_estimator=CatBoostClassifier(n_estimators=500, silent=True),
    cv=5,
    n_jobs=1
)
final_model = MultiOutputClassifier(estimator=stacked_model, n_jobs=-1)

In [24]:
# final_model.fit(X_train, y_train)
final_model.fit(train, target)

/opt/miniconda3/lib/python3.9/site-packages/sklearn/pipeline.py:348: UserWarning: Persisting input arguments took 3.42s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  X, fitted_transformer = fit_transform_one_cached(
/opt/miniconda3/lib/python3.9/site-packages/sklearn/pipeline.py:348: UserWarning: Persisting input arguments took 3.44s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  X, fitted_transf

KeyboardInterrupt: 

In [ ]:
# pred_proba = final_model.predict_proba(X_test)
# pred_proba = np.stack([p[:, 1] for p in pred_proba]).T
# pred_labels_opt = get_pred_labels(pred_proba)
# score = f1_score(y_test, pred_labels_opt, average='samples')
# print(score)

0.8263101243904494


In [ ]:
# 0.8067963921465854 - lgb
# 0.8006289720238485 - LinSVC
# 0.8073270872917077 - LinSVC + WOE
# 0.822465470496657 - LinSVC + WOE + param treshold
# 0.8224286556658961 - LinSVC + WOE + GLMMEncoder(ORDERED_COLS) + param treshold 
# 0.8247302065686393 - add manual features
# 0.8247876127454192 - add target & count encoders
# 0.8263101243904494 - add morph features

In [ ]:
# TARGET_COL = 7
# X_test.loc[(X_test.iloc[:, TARGET_COL].values != pred_labels_opt[:, TARGET_COL]) & (X_test.iloc[:, TARGET_COL] == 1), 'negative'].tolist()

['Для начала, наймите грамотного рекрутера, который будет адекватно оценивать соискателей на открытые вакансии !',
 'Много обязанностей, маленькая зп Ты там и кассир и продавец и уборщица и охранник и мерчендайзер',
 'Условия оплаты и труда',
 'Улучшить нужно все начиная с работников склада, заканчивая руководством.',
 'Все, начиная от условий на рабочем месте, т. е. В броневике (кондиционер например, ибо стекла там не опускаются и летом температура достигает 50°), зарплатой которая у инкассатора без переработки чистыми выходила 19 000р а с переработкой в 40 часов в районе 25 000р и заканчивая тем что, начальник 8 участка никогда не брал на себя ответственность за свои решения, а всегда делал крайним любого другого',
 '.............. ..........',
 'Повысить зарплату, нанять грузчиков, уборщиц, составить чёткие планограммы, Оплачивать рабочие праздничные дни в двойном размере, как положено по закону. Оплачивать выход в свой выходной в двойном размере',
 'Дать сотрудникам возможность офо

In [ ]:
# TARGET_COL = 8
# X_test.loc[(X_test.iloc[:, TARGET_COL].values != pred_labels_opt[:, TARGET_COL]) & (X_test.iloc[:, TARGET_COL] == 1), 'negative'].tolist()

['.....................',
 'Ввести минимальный оклад, бесплатные звонки за счёт компании, перестать набирать ораву людей, когда нет заказов, воскресить Сталина',
 'Отпишите меня от этих рассылок, вы уже достали кошмарить меня своими сообщениями о ваших "прекрасных" скидках и предложениях. Каждый день наяривают свои сообщения, а иногда и не по разу в день. Сколько раз еще надо написать, позвонить, чтоб вы поняли? Итак не часто хожу в ваш магазин, но С таким отношением я принципиально буду обходить ваш магазин стороной!!',
 'Хоть отвечать научитесь',
 'Улучшить безусловно нужно компетенции непосредственно исполняющего обязанности директора. Человек абсолютно некомпетентен в организации рабочего процесса на производстве. Трудозатраты распределены безраздумно и нелогично. Советы и критика не воспринимаются от слова совсем. Выводы можно сделать проанализировав текучку руководящего аппарата на производстве. Люди работают без перерыва на обед, с задержками после рабочего времени, и никак это 

In [ ]:
# TARGET_COL = 6
# X_test.loc[(X_test.iloc[:, TARGET_COL].values != pred_labels_opt[:, TARGET_COL]) & (X_test.iloc[:, TARGET_COL] == 1), 'negative'].tolist()

['Для начала, наймите грамотного рекрутера, который будет адекватно оценивать соискателей на открытые вакансии !',
 'Много обязанностей, маленькая зп Ты там и кассир и продавец и уборщица и охранник и мерчендайзер',
 'Улучшить нужно все начиная с работников склада, заканчивая руководством.',
 'Все, начиная от условий на рабочем месте, т. е. В броневике (кондиционер например, ибо стекла там не опускаются и летом температура достигает 50°), зарплатой которая у инкассатора без переработки чистыми выходила 19 000р а с переработкой в 40 часов в районе 25 000р и заканчивая тем что, начальник 8 участка никогда не брал на себя ответственность за свои решения, а всегда делал крайним любого другого',
 '.............. ..........',
 'Повысить зарплату, нанять грузчиков, уборщиц, составить чёткие планограммы, Оплачивать рабочие праздничные дни в двойном размере, как положено по закону. Оплачивать выход в свой выходной в двойном размере',
 'обучение.. его не хватает для новых сотрудников, также прис

In [ ]:
# TARGET_COL = 5
# X_test.loc[(X_test.iloc[:, TARGET_COL].values != pred_labels_opt[:, TARGET_COL]) & (X_test.iloc[:, TARGET_COL] == 1), 'negative'].tolist()

['Для начала, наймите грамотного рекрутера, который будет адекватно оценивать соискателей на открытые вакансии !',
 'Много обязанностей, маленькая зп Ты там и кассир и продавец и уборщица и охранник и мерчендайзер',
 'Улучшить нужно все начиная с работников склада, заканчивая руководством.',
 'Повысить зарплату, нанять грузчиков, уборщиц, составить чёткие планограммы, Оплачивать рабочие праздничные дни в двойном размере, как положено по закону. Оплачивать выход в свой выходной в двойном размере',
 'Внедрение в коллектив команды. Знакомство с устоявшимися процессами и негласными правилами',
 'Всё! И в кротчайшие сроки! Ибо это потери',
 'При трудоустройстве говорят что зарплата складывается из оклада и сделки, по факту сдельная часть относится к премии и выплачивается на усмотрение работодателя. Грубо говоря получаешь 12000 рублей как оклад, а остальное они могут не платить.',
 'Сбалансировать работу- оплату',
 'Необходимо думать о людях и платить им',
 'Да Я твьядчддсьяиярлцьчтчодвт',


In [ ]:
# TARGET_COL = 4
# X_test.loc[(X_test.iloc[:, TARGET_COL].values != pred_labels_opt[:, TARGET_COL]) & (X_test.iloc[:, TARGET_COL] == 1), 'negative'].tolist()

['Для начала, наймите грамотного рекрутера, который будет адекватно оценивать соискателей на открытые вакансии !',
 'Много обязанностей, маленькая зп Ты там и кассир и продавец и уборщица и охранник и мерчендайзер',
 'Условия оплаты и труда',
 'Улучшить нужно все начиная с работников склада, заканчивая руководством.',
 'Все, начиная от условий на рабочем месте, т. е. В броневике (кондиционер например, ибо стекла там не опускаются и летом температура достигает 50°), зарплатой которая у инкассатора без переработки чистыми выходила 19 000р а с переработкой в 40 часов в районе 25 000р и заканчивая тем что, начальник 8 участка никогда не брал на себя ответственность за свои решения, а всегда делал крайним любого другого',
 '.............. ..........',
 'Повысить зарплату, нанять грузчиков, уборщиц, составить чёткие планограммы, Оплачивать рабочие праздничные дни в двойном размере, как положено по закону. Оплачивать выход в свой выходной в двойном размере',
 'обучение.. его не хватает для но

In [ ]:
# pred_proba_train = cross_val_predict(
#     estimator=clone(final_model),
#     X=train,
#     y=target,
#     cv=3,
#     verbose=5,
#     n_jobs=1,
#     method="predict_proba"
# )

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/opt/miniconda3/lib/python3.9/site-packages/sklearn/pipeline.py:348: UserWarning: Persisting input arguments took 2.02s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  X, fitted_transformer = fit_transform_one_cached(
/opt/miniconda3/lib/python3.9/site-packages/sklearn/pipeline.py:348: UserWarning: Persisting input arguments took 2.18s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the jo

In [ ]:
# pred_proba_train = np.stack([p[:, 1] for p in pred_proba_train]).T
# pred_labels_opt_train = get_pred_labels(pred_proba_train)
# score = f1_score(target, pred_labels_opt_train, average='samples')
# print(score)

0.8253868415830833


In [ ]:
# 0.8253868415830833 - add target & count encoders